In [1]:

from bs4 import BeautifulSoup
import requests, json


In [2]:

r0 = requests.get("https://www.imdb.com/feature/genre/")
r0.status_code


200

In [3]:
soup = BeautifulSoup(r0.content, "html.parser")


In [4]:

selector ="image"
res0 = soup.find_all("div", {"class": selector})


In [5]:
hyperlinks, titles = [], []

for item in res0:
    hyperlinks.append(item.a['href'])
    titles.append(item.a.img['title'])

In [6]:

r1 = requests.get(hyperlinks[0])
type(r1.status_code)


int

In [7]:

def scraper( bu):
    
    tr = requests.get(bu)
    if tr.status_code != 200:
        return []
    
    ts = BeautifulSoup(tr.content, "html.parser")
        
    selector = "lister-item-content"
    tres = ts.find_all("div", {"class": selector})    
    
    ms_, yr_, rt_ = [], [], []
        
    for item in tres:
        ms_.append(item.a.string)
    
    tres2 = ts.find_all("div", class_="ratings-bar")
    for item in tres2:
        rt_.append(item.strong.string)
    
            
    selector = "lister-item-year text-muted unbold"
    tres3 = ts.find_all("span", class_=selector)

    for item in tres3:
        yr_.append(item.string)

    return (ms_, yr_, rt_ )   


In [8]:

result = {}

for item in range(len(titles)):
    
    ms_, yr_, rt_ = scraper(hyperlinks[item])
    
    result[titles[item]] = {
        "movie_series": ms_,
        "year_release": yr_,
        "rating": rt_,
        "base_url": hyperlinks[item]
    }


In [9]:

import numpy as np
import pandas as pd


In [10]:
df1 = pd.DataFrame(result)

In [11]:
# type(df1["Sci-Fi"])

In [12]:
final_df = []


for item in titles:
        
    tseries = pd.Series(df1[item]["movie_series"], name="movie_series")
    tyear = pd.Series(df1[item]['year_release'], name="year_release")
    trating = pd.Series(df1[item]["rating"], name="rating")
    
    tmp_df = pd.concat([tseries, tyear, trating], axis=1)
    final_df.append(tmp_df)
    


In [13]:
def file_creator():
    
    for idx in range(len(final_df)):
        filename = f"{titles[idx]}.csv"
        
        tdf = final_df[idx]
        tdf.to_csv(
            filename
        )
        

file_creator()     


In [51]:

import mysql.connector as mysql
from mysql.connector import Error

try:
    con = mysql.connect(
        host="127.0.0.1",
        user="prince",
        password="devil@db007"
    )
    
    if con.is_connected():
        cursor = con.cursor()
        res = cursor.execute("CREATE DATABASE IF NOT EXISTS movies")
        
except Error as error:
    print("Error while connecting to MYSQL")


In [58]:

def insert_data(
    data_frame, db_name, table_name
):
    
    if '-' in table_name:
        new_tbname = table_name.split("-")
        table_name = "_".join(new_tbname)
    
    try:
        if con.is_connected():
            
            cursor = con.cursor()
            cursor.execute(f"use {db_name}")
            
            record = cursor.fetchone()
            print("Connection Established With {}".format(record))
            
            cursor.execute("DROP TABLE IF EXISTS {}".format(table_name))
            print("Creating Table {}...".format(table_name))
            
            cursor.execute(
                "CREATE TABLE IF NOT EXISTS {a} ( {a}_id INT AUTO_INCREMENT, movie_series VARCHAR(255) NOT NULL, rating FLOAT(2,1), year_release VARCHAR(255), PRIMARY KEY({a}_id) )".format(a=table_name)
            )
            
            print("Table Created {}...".format(table_name))
            
            data_frame["rating"] = data_frame["rating"].astype(np.float16)
            
            if data_frame["rating"].hasnans:
                data_frame = data_frame.fillna(np.mean(
                    data_frame["rating"]
                ))
            
            queries= []
            for i in range(len(data_frame)):
        
                val1, val2, val3 = data_frame.iloc[i]["movie_series"], data_frame.iloc[i]["rating"], data_frame.iloc[i]["year_release"]
                sql = """
                    INSERT INTO {a}.{b} (movie_series, rating, year_release)
                    VALUES
                    ("{v1}", {v2}, "{v3}")
                """.format(a=db_name, b=table_name, v1=val1, v2=val2, v3=val3)
            
                queries.append(sql)

            for query in queries:
                cursor.execute(query)
                con.commit()
    except Error as error:
        print("Error :" + str(error))


In [53]:
final_df[1].sample(3)
# print(titles[1])

,movie_series,year_release,rating
41,Mighty Morphin Power Rangers,(1993–1996),6.4
48,The Big Door Prize,(2023– ),NaN
31,Star Wars: The Clone Wars,(2008–2020),8.0


In [54]:

while item < len(final_df):
    insert_data(final_df[item], db_name="movies", table_name=str(titles[item]))
    item += 1
    print(item)


In [55]:

def query_db(
    db_name, table_name
):
    
    if '-' in table_name:
        new_tbname = table_name.split("-")
        table_name = "_".join(new_tbname)
    
    sql = 'SELECT * FROM {}.{}'.format(db_name, table_name)
    cursor.execute(sql)
    
    result = cursor.fetchall()
    for i in result:
        print(i)



In [60]:
def close_connection(con=None):
    if not con:
        return
    con.close()

In [64]:
try:
    con2 = mysql.connect(
        host="localhost",
        user="prince",
        password="devil@db007",
    )
    
    if con2.is_connected():
        cursor = con2.cursor()
        cursor.execute("CREATE DATABASE IF NOT EXISTS tmp_db0")
        print("database created ...")
        
        cursor.execute("CREATE TABLE IF NOT EXISTS tmp_db0.curr_table (id INT, name VARCHAR(255), rating FLOAT, year_release VARCHAR(255))")
        print("table created ...")
except Error as error:
    print(str(error))

database created ...
table created ...


In [66]:

final_df[0][:5]


,movie_series,year_release,rating
0,Succession,(2018–2023),8.796875
1,Ghosted,(I) (2023),5.800781
2,Beef,(2023– ),8.203125
3,Ted Lasso,(2020– ),8.796875
4,The Super Mario Bros. Movie,(2023),7.300781
